#Run .py files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/redoio/three_strikes_project/

In [ ]:
!python three_strikes_project/resentencing_data_initiative/code/run.py

#Run .pynb file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [ ]:
# Set parameters
county_name = 'Los Angeles'
month = '/'.join(['Rough', 'Data_05_2021', '21_05'])
data_path = '/content/drive/My Drive/Stanford Law 3XP/Data'
code_path = '/content/drive/My Drive/Stanford Law 3XP/Code'

#Sorting Criteria

In [ ]:
county_sort_criteria_sheet = '/'.join([data_path, county_name, 'Criteria/sorting_criteria.xlsx'])

In [ ]:
sorting_criteria = pd.read_excel(county_sort_criteria_sheet)

In [ ]:
sorting_criteria.head()

#Demographics

In [ ]:
county_demographics_sheet = '/'.join([data_path, county_name, month, 'demographics.xlsx'])

In [ ]:
# Load the data
demographics = pd.read_excel(county_demographics_sheet)

In [ ]:
demographics.head()

Visualizing demographics data

In [ ]:
print('Demographics by ethnicity')
demographics[demographics['Ethnicity'].isin(['Black', 'White', 'Hispanic', 'Mexican', 'Other'])]['Ethnicity'].hist()

In [ ]:
print('Demographics by ethnicity')
demographics[~demographics['Ethnicity'].isin(['Black', 'White', 'Hispanic', 'Mexican', 'Other'])]['Ethnicity'].hist(xrot = 90)

# Individual Details

In [ ]:
# Education merit
merit_credit_sheet = '/'.join([data_path, county_name, month, 'EducationMeritCredits.xlsx'])
merit_credit = pd.read_excel(merit_credit_sheet)
# Milestone credit
milestone_credit_sheet = '/'.join([data_path, county_name, month, 'MilestoneCompletionCredits.xlsx'])
milestone_credit = pd.read_excel(milestone_credit_sheet)
# Rehab credit
rehab_credit_sheet = '/'.join([data_path, county_name, month, 'RehabilitiveAchievementCredits.xlsx'])
rehab_credit = pd.read_excel(rehab_credit_sheet)
# Vocational education credit
voced_sheet = '/'.join([data_path, county_name, month, 'VocEd_TrainingCerts.xlsx'])
voced_credit = pd.read_excel(voced_sheet)
# Rule violations
rvr_sheet = '/'.join([data_path, county_name, month, 'RVRs.xlsx'])
rv_report = pd.read_excel(rvr_sheet)

#Current Commitments

In [ ]:
county_current_commits_sheet = '/'.join([data_path, county_name, month, 'currentcommitments.xlsx'])

In [ ]:
current_commits = pd.read_excel(county_current_commits_sheet)

In [ ]:
current_commits.head()

#Prior Commitments

In [ ]:
county_prior_commits_sheet = '/'.join([data_path, county_name, month, 'priorcommitments.xlsx'])

In [ ]:
prior_commits = pd.read_excel(county_prior_commits_sheet)

In [ ]:
prior_commits.head()

## Functions

In [ ]:
# Functions

def gen_time_vars(df):
  # Get the present date
  present_date = datetime.datetime.now()
  # Sentence duration in years
  df['Aggregate sentence in years'] = df['Aggregate Sentence in Months']/12
  # Age of individual
  df['Age in years'] = [x.days/365 for x in present_date - pd.to_datetime(demographics['Birthday'], errors = 'coerce')]
  # Sentence served in years
  df['Time served in years'] = [x.days/365 for x in present_date - pd.to_datetime(demographics['Offense End Date'], errors = 'coerce')]
  # Age at the time of offense
  df['Age during offense'] = [x.days/365 for x in pd.to_datetime(demographics['Offense End Date'], errors = 'coerce') - pd.to_datetime(demographics['Birthday'], errors = 'coerce')]

  # Store all data that have NaNs for any of the time columns calculated above
  def incorrect_time(df, cols = ['Aggregate sentence in years', 'Age in years', 'Time served in years', 'Age during offense']):
    errors = pd.DataFrame()
    # Loop through all time columns
    for col in cols:
      errors = pd.concat([errors, df[pd.isna(df[col])]])
    return errors

  # Return the resulting dataframe with the calculated time columns and the data with NaN/NaTs in these columns
  return df, incorrect_time(df)


def clean_offense(off):
  # Lowercase all letters
  clean_off = str(off).lower()
  # Remove trailing periods
  clean_off = clean_off.rstrip('.')
  # Remove whitespace (any location)
  clean_off = clean_off.replace(' ', '')
  # Remove "PC" or penal code abbreviation
  clean_off = clean_off.replace('pc', '')
  # Remove "rape" which shows up in some offenses
  clean_off = clean_off.replace('rape', '')
  return clean_off


def clean_offense_blk(data):
  # If input is a single string
  if isinstance(data, str):
    return clean_offense(data)
  # If input is a list of strings
  elif isinstance(data, list):
    off_clean = []
    for off in data:
      off_clean.append(clean_offense(off))
    return off_clean
  # If input is a column of a pandas dataframe
  elif isinstance(data, pd.Series):
    return data.apply(clean_offense)


def gen_inel_off(inel_offenses, clean = True, impl = {'all': ["/att", "(664)", "2nd"], '459': ["/att", "(664)"]}, perm = 2):
  # Clean the offense data if specified
  if clean:
    inel_offenses = clean_offense_blk(inel_offenses)

  def gen_impl_off():
    # Generate new list of offenses based on the implied ineligibility
    add = []
    # Loop through all offenses in the ineligible offenses list
    for off in inel_offenses:
      # Check the two conditions: generic or exception
      matching = [key for key in impl.keys() if key in off]
      # If offense is not called out separately (exception)
      if (len(matching) == 0) and ('all' in impl.keys()):
        for impl_val in impl['all']:
          # If any additions are not already in the offense, ex: PC 123(664) does not need PC 123(664)(664) to be added
          if impl_val not in off:
            add.append(off+impl_val)
      # If offense is called out separately (exception)
      elif len(matching) != 0:
        for impl_val in impl[matching[0]]:
          # If any additions are not already in the offense, ex: PC 123(664) does not need PC 123(664)(664) to be added
          if impl_val not in off:
            add.append(off+impl_val)
    # Combine newly identified ineligible offenses to the list of existing ineligible offenses and return result
    return list(set.union(set(inel_offenses), set(add)))

  # Generate permutations of the ineligible offenses
  i = 1
  while i <= perm:
    # Run the function to generate implied ineligibility
    inel_offenses = gen_impl_off()
    i = i + 1

  # Return the final results after all permutations
  return inel_offenses


def det_inel_off(offenses, inel_offenses, pop = None):
    # Return offenses that are ineligible for adults and juveniles
    if pop == 'adult' or pop == 'juvenile':
      return set(offenses).intersection(set(inel_offenses))
    # If none of the conditions are met
    else:
      print('No offenses processed. Please double check inputs and re-run')

#Conditions for Qualification

Cohort 1

Adults



1.   Age 50 and older; AND
2.   Sentenced to 20 years or more; AND
3.   Served a minimum of 10 years in custody; AND
4.   Is not serving a current sentence for any offense listed in Table A, B, C, or D, AND
5.   Does not have a prior conviction for any offense listed in Tables C & D

Cohort 2

Minors Tried as Adults

1.   Sentenced for a crime that was committed at age 14 or 15; AND
2.   Not serving current sentence for any offense listed in Table D and E; AND
3. Has served a minimum of 10 years in custody; AND
4. Does not have a prior conviction for any offense listed in Table D.

# Time Data

In [ ]:
# Add all of the time variables to the demographic data necessary for classification - years served, sentence length, age, etc.
demographics, errors = gen_time_vars(demographics)

In [ ]:
# Show CDCR numbers and data of individuals with no age or time-served information
errors

#Identify Candidates: Adults (Part 1/4)

Conditions to search:
1., 2. and 3. for Cohort 1

In [ ]:
print('Total number of CDCR#s available: ', len(demographics['CDCR #'].unique()))

# Extracting CDCR numbers with eligible ages
el_cdcr_nums_1 = demographics[demographics['Age in years'] >= 50]['CDCR #']
print('Number of CDCR#s that are older than 50 years: ', len(el_cdcr_nums_1))

# Extracting CDCR numbers that met the age criteria that also meet the time sentenced criteria
el_cdcr_nums_2 = demographics[(demographics['Aggregate sentence in years'] >= 20) & demographics['CDCR #'].isin(el_cdcr_nums_1)]['CDCR #']
print('Number of CDCR#s that are older than 50 years & sentenced to over 20 years: ', len(el_cdcr_nums_2))

# Extracting CDCR numbers that met the age criteria that also meet the time served criteria
el_cdcr_nums_3 = demographics[(demographics['Time served in years'] >= 10) & demographics['CDCR #'].isin(el_cdcr_nums_2)]['CDCR #']
print('Number of CDCR#s that are older than 50 years, sentenced to over 20 years and served over 10 years: ', len(el_cdcr_nums_3))

#Identify Candidates: Adults (Part 2/4)

Conditions to search:

4. and 5. for Cohort 1



In [ ]:
# Extracting ineligible offenses from sorting criteria
inel_offenses = sorting_criteria[sorting_criteria['Table'].isin(['Table A', 'Table B', 'Table C', 'Table D'])]['Offenses'].tolist()
# Appending new offenses based on implied ineligibility for adult populations
inel_offenses = gen_inel_off(inel_offenses, clean = True, impl = {'all': ["/att", "(664)", "2nd"], '459': ["/att", "(664)"]})

# Clean offense data in current commits file
current_commits['Offense cleaned'] = clean_offense_blk(data = current_commits['Offense'])

# Extracting current commits data with eligible offenses
el_cdcr_nums_4 = []
for cdcr_num in tqdm(el_cdcr_nums_3):
  # Extract offenses of the CDCR number
  offenses = current_commits[current_commits['CDCR #'] == cdcr_num]['Offense cleaned'].unique()
  if len(det_inel_off(offenses = offenses, inel_offenses = inel_offenses, pop = 'adult')) == 0:
    el_cdcr_nums_4.append(cdcr_num)

print('\nNumber of CDCR#s that are older than 50 years, sentenced to over 20 years, served over 10 years, and have eligible current offenses: ', len(el_cdcr_nums_4))

In [ ]:
# Extracting ineligible offenses from sorting criteria
inel_offenses = sorting_criteria[sorting_criteria['Table'].isin(['Table C', 'Table D'])]['Offenses'].tolist()
# Appending new offenses based on implied ineligibility for adult populations
inel_offenses = gen_inel_off(inel_offenses, clean = True, impl = {'all': ["/att", "(664)", "2nd"], '459': ["/att", "(664)"]})

# Clean offense data in prior commits file
prior_commits['Offense cleaned'] = clean_offense_blk(data = prior_commits['Offense'])

# Extracting prior commits data with eligible offenses
el_cdcr_nums_5 = []
for cdcr_num in tqdm(el_cdcr_nums_4):
  # Extract offenses of the CDCR number
  offenses = prior_commits[prior_commits['CDCR #'] == cdcr_num]['Offense cleaned'].unique()
  if len(det_inel_off(offenses = offenses, inel_offenses = inel_offenses, pop = 'adult')) == 0:
    el_cdcr_nums_5.append(cdcr_num)

print('\nNumber of CDCR#s that are older than 50 years, sentenced to over 20 years, served over 10 years, have eligible current offenses and eligible prior offenses: ', len(el_cdcr_nums_5))

### Analyzing individuals in Cohort 1 who meet all 5 conditions

Writing results to Excel files

In [ ]:
# Write data to excel files
write_path = '/'.join([data_path, county_name, month, 'adult_eligible_demographics.xlsx'])
demographics[demographics['CDCR #'].isin(el_cdcr_nums_5)].to_excel(write_path, index = False)

In [ ]:
# Write data to excel files
write_path = '/'.join([data_path, county_name, month, 'adult_eligible_currentcommits.xlsx'])
current_commits[current_commits['CDCR #'].isin(el_cdcr_nums_5)].to_excel(write_path, index = False)

Validation of Cohort 1 results with OpenLattice results

In [ ]:
ol_el_cdcr_nums = pd.read_excel('/'.join([data_path, county_name, 'Rough/LA_DA_Cohort1_Update_05_2021.xlsx']))['CDCR..'].to_list()

In [ ]:
# Find CDCR numbers eligible in OpenLattice script that are ineligible in this script
missing_nums = []
for cdcr_num in ol_el_cdcr_nums:
  if (cdcr_num in demographics['CDCR #'].tolist()) and (cdcr_num not in el_cdcr_nums_5):
    missing_nums.append(cdcr_num)

In [ ]:
# Missing CDCR numbers
d = {}
write_path = '/'.join([data_path, county_name, 'Rough', 'ol_validation_.xlsx'])
for cdcr_num in missing_nums:
  off = current_commits[current_commits['CDCR #'] == cdcr_num]['Offense']
  d[cdcr_num] = off.to_list()
print('These CDCR numbers are eligible according to OpenLattice script but are ineligible according to this script')

df = pd.DataFrame()
df['CDCR #'] = d.keys()
df['Offenses'] = d.values()
df.to_excel(write_path, index = False)

In [ ]:
# Find CDCR numbers ineligible in this script that are eligible in OpenLattice script
missing_nums = []
for cdcr_num in el_cdcr_nums_5:
  if cdcr_num not in ol_el_cdcr_nums:
    missing_nums.append(cdcr_num)

In [ ]:
# Missing CDCR numbers
for cdcr_num in missing_nums:
  off = current_commits[current_commits['CDCR #'] == cdcr_num]['Offense']
  print(cdcr_num, ':', off.to_list(), ';')
print('These CDCR numbers are eligible according to this script but are ineligible according to OpenLattice script')

Individual details

In [ ]:
# Write data to excel files
write_path = '/'.join([data_path, county_name, month, 'summary_adult.xlsx'])

In [ ]:
# Get demographics data
df = demographics.loc[demographics['CDCR #'].isin(el_cdcr_nums_5)][['CDCR #', 'Current Security Level', 'Controlling Offense',
                                                                    'Current Classication Score', 'Classification Score 5 Years\nAgo',
                                                                    'Mental Health Level of Care', 'DPPV Disability - Mobility']]

# Remove new-line
df.rename(columns = {'Classification Score 5 Years\nAgo': 'Classification Score 5 Years Ago'}, inplace = True)
rv_report.rename(columns = {'Rule\nViolation\nDate': 'Rule Violation Date'}, inplace = True)

# Format mobility disability
df['DPPV Disability - Mobility'] = df['DPPV Disability - Mobility'].str.replace('Impacting Placement', '')

# Initialize lists for other variables
current_conv = []
prior_conv = []
programming = []
rvr = []
for cdcr_num in df['CDCR #']:
  # Current convictions
  current_conv.append(', '.join(current_commits[current_commits['CDCR #'] == cdcr_num]['Offense'].tolist()))
  # Previous convictions
  prior_conv.append(', '.join(prior_commits[prior_commits['CDCR #'] == cdcr_num]['Offense'].tolist()))
  # Participation in programming
  if (cdcr_num in merit_credit['Cdcno']) or (cdcr_num in milestone_credit['Cdcno']) or (cdcr_num in rehab_credit['Cdcno']) or (cdcr_num in voced_credit['Cdcno']):
    programming.append('Yes')
  else:
    programming.append('No')
  # Rule violation reports
  ext = rv_report[rv_report['CDCR\nNumber'] == cdcr_num][['Rule Violation Date', 'Division', 'Rule Violation']].reset_index(drop = True).to_dict('index')
  rvr.append("\n\n".join("\n".join(k_b + ': ' + str(v_b) for k_b, v_b in v_a.items()) for k_a, v_a in ext.items()))

# Store lists in dataframe
df['Current Convictions'] = current_conv
df['Prior Convictions'] = prior_conv
df['Programming'] = programming
df['Rules Violations'] = rvr

# Write data to excel files
df.to_excel(write_path, index = False)

Exploring Cohort 1 offenses, racial makeup, penal codes



In [ ]:
print('Top 20 offenses of individuals in Cohort 1 who meet all 5 eligibility conditions (from demographics data)')
print(demographics[demographics['CDCR #'].isin(el_cdcr_nums_5)]['Description'].value_counts()[0:20])

In [ ]:
print('Top 20 offenses of individuals in Cohort 1 who meet all 5 eligibility conditions (from demographics data)')
demographics[demographics['CDCR #'].isin(el_cdcr_nums_5)]['Controlling Offense'].value_counts()[0:20]

In [ ]:
print('Top 20 offenses of individuals in Cohort 1 who meet all 5 eligibility conditions (from current commits data)')
current_commits[current_commits['CDCR #'].isin(el_cdcr_nums_5)]['Offense'].value_counts()[0:20]

In [ ]:
print('Sex offenses of individuals in Cohort 1 who meet all 5 eligibility conditions')
print(demographics[demographics['CDCR #'].isin(el_cdcr_nums_5)]['Sex Registrant'].value_counts())

In [ ]:
print('Type of offenses of individuals in Cohort 1 who meet all 5 eligibility conditions')
print(demographics[demographics['CDCR #'].isin(el_cdcr_nums_5)]['Offense Category'].value_counts())

In [ ]:
demographics[(demographics['Ethnicity'].isin(['Black', 'White', 'Hispanic', 'Mexican', 'Other'])) & (demographics['CDCR #'].isin(el_cdcr_nums_5))]['Ethnicity'].hist()
print('Ethnicities of the individuals in Cohort 1 who meet all 5 eligibility criteria')

In [ ]:
demographics[(~demographics['Ethnicity'].isin(['Black', 'White', 'Hispanic', 'Mexican', 'Other'])) & (demographics['CDCR #'].isin(el_cdcr_nums_5))]['Ethnicity'].hist(xrot = 90)
print('Ethnicities of the individuals in Cohort 1 who meet all 5 eligibility criteria')

#Conditions for Qualification

Cohort 1

Adults



1.   Age 50 and older; AND
2.   Sentenced to 20 years or more; AND
3.   Served a minimum of 10 years in custody; AND
4.   Is not serving a current sentence for any offense listed in Table A, B, C, or D, AND
5.   Does not have a prior conviction for any offense listed in Tables C & D

Cohort 2

Minors Tried as Adults

1.   Sentenced for a crime that was committed at age 14 or 15; AND
2.   Not serving current sentence for any offense listed in Table E or D; AND
3. Has served a minimum of 10 years in custody; AND
4. Does not have a prior conviction for any offense listed in Table D.

# Identify Candidates: Juvenile (Part 3/4)

Conditions to search:

  1., 2. for Cohort 2



In [ ]:
# Extracting CDCR numbers that meet the age criteria
el_cdcr_nums_1 = []
el_cdcr_nums_1 = demographics[(demographics['Age during offense'] < 16) & (demographics['Age during offense'] >= 14)]['CDCR #'].to_list()
print('Number of CDCR#s that committed offenses at the age of 14 to 15: ', len(el_cdcr_nums_1))

In [ ]:
# Extracting ineligible offenses from sorting criteria
inel_offenses = sorting_criteria[sorting_criteria['Table'].isin(['Table E', 'Table D'])]['Offenses'].tolist()
inel_offenses = gen_inel_off(inel_offenses, clean = True, impl = {'187': ["2nd", "(664)"]}, perm = 2)

# Clean offense data in current commits file
current_commits['Offense cleaned'] = clean_offense_blk(data = current_commits['Offense'])

# Extracting CDCR numbers that meet the age criteria and offense eligibility
el_cdcr_nums_2 = []
for cdcr_num in tqdm(el_cdcr_nums_1):
  # Extracting offenses of the CDCR number
  offenses = current_commits[current_commits['CDCR #'] == cdcr_num]['Offense cleaned'].unique()
  if len(det_inel_off(offenses = offenses, inel_offenses = inel_offenses, pop = 'juvenile')) == 0:
    el_cdcr_nums_2.append(cdcr_num)

print('\nNumber of CDCR#s that committed offenses at age 14 and 15 and have eligible current offenses: ', len(el_cdcr_nums_2))

# Identify Candidates: Juvenile (Part 4/4)

Conditions to search:
3. and 4. for Cohort 2

In [ ]:
# Extracting CDCR numbers that met the age and offense criteria that also meet the time served criteria
el_cdcr_nums_3 = []
el_cdcr_nums_3 = demographics[(demographics['Time served in years'] >= 10) & demographics['CDCR #'].isin(el_cdcr_nums_2)]['CDCR #'].to_list()
print('Number of CDCR#s that committed offenses at age 14 and 15, have eligible current offenses and served more than 10 years: ', len(el_cdcr_nums_3))

In [ ]:
# Extracting ineligible offenses from sorting criteria
inel_offenses = sorting_criteria[sorting_criteria['Table'].isin(['Table D'])]['Offenses'].tolist()
inel_offenses = gen_inel_off(inel_offenses, clean = True, impl = {'187': ["2nd", "(664)"]}, perm = 2)

# Clean offense data in current commits file
prior_commits['Offense cleaned'] = clean_offense_blk(data = prior_commits['Offense'])

# Extracting CDCR numbers that met the age, time sentenced and current and prior offense eligibility criteria
el_cdcr_nums_4 = []
for cdcr_num in tqdm(el_cdcr_nums_3):
  offenses = prior_commits[prior_commits['CDCR #'] == cdcr_num]['Offense cleaned'].unique()
  if len(det_inel_off(offenses = offenses, inel_offenses = inel_offenses, pop = 'juvenile')) == 0:
    el_cdcr_nums_4.append(cdcr_num)

print('\nNumber of CDCR#s that committed offenses at age 14 and 15, have eligible current and prior offenses, and served more than 10 years: ', len(el_cdcr_nums_4))

Individuals in Cohort 2 who meet all the eligibility criteria

In [ ]:
# Write data to excel files
write_path = '/'.join([data_path, county_name, month, 'juvenile_eligible_demographics.xlsx'])
demographics[demographics['CDCR #'].isin(el_cdcr_nums_4)].to_excel(write_path, index = False)

In [ ]:
# Write data to excel files
write_path = '/'.join([data_path, county_name, month, 'juvenile_eligible_currentcommits.xlsx'])
current_commits[current_commits['CDCR #'].isin(el_cdcr_nums_4)].to_excel(write_path, index = False)

Exploring Cohort 1 offenses, racial makeup, penal codes


In [ ]:
print('Top 20 offenses of individuals in Cohort 2 who meet all 4 eligibility conditions (from demographics data)')
print(demographics[demographics['CDCR #'].isin(el_cdcr_nums_4)]['Description'].value_counts()[0:20])

In [ ]:
print('Top 20 offenses of individuals in Cohort 2 who meet all 4 eligibility conditions (from demographics data)')
demographics[demographics['CDCR #'].isin(el_cdcr_nums_4)]['Controlling Offense'].value_counts()[0:20]

In [ ]:
print('Sex offenses of individuals in Cohort 2 who meet all 4 eligibility conditions')
print(demographics[demographics['CDCR #'].isin(el_cdcr_nums_4)]['Sex Registrant'].value_counts())

In [ ]:
print('Type of offenses of individuals in Cohort 2 who meet all 4 eligibility conditions')
print(demographics[demographics['CDCR #'].isin(el_cdcr_nums_4)]['Offense Category'].value_counts())

In [ ]:
demographics[(demographics['Ethnicity'].isin(['Black', 'White', 'Hispanic', 'Mexican', 'Other'])) & (demographics['CDCR #'].isin(el_cdcr_nums_4))]['Ethnicity'].hist()
print('Ethnicities of the individuals in Cohort 2 who meet all 4 eligibility criteria')

Generate summary


In [ ]:
# Write data to excel files
write_path = '/'.join([data_path, county_name, month, 'summary_juvenile.xlsx'])

In [ ]:
# Get demographics data
df = demographics.loc[demographics['CDCR #'].isin(el_cdcr_nums_4)][['CDCR #', 'Current Security Level', 'Controlling Offense',
                                                                    'Current Classication Score', 'Classification Score 5 Years\nAgo',
                                                                    'Mental Health Level of Care', 'DPPV Disability - Mobility']]

# Remove new-line
df.rename(columns = {'Classification Score 5 Years\nAgo': 'Classification Score 5 Years Ago'}, inplace = True)
rv_report.rename(columns = {'Rule\nViolation\nDate': 'Rule Violation Date'}, inplace = True)

# Format mobility disability
df['DPPV Disability - Mobility'] = df['DPPV Disability - Mobility'].str.replace('Impacting Placement', '')

# Initialize lists for other variables
current_conv = []
prior_conv = []
programming = []
rvr = []
for cdcr_num in df['CDCR #']:
  # Current convictions
  current_conv.append(', '.join(current_commits[current_commits['CDCR #'] == cdcr_num]['Offense'].tolist()))
  # Previous convictions
  prior_conv.append(', '.join(prior_commits[prior_commits['CDCR #'] == cdcr_num]['Offense'].tolist()))
  # Participation in programming
  if (cdcr_num in merit_credit['Cdcno']) or (cdcr_num in milestone_credit['Cdcno']) or (cdcr_num in rehab_credit['Cdcno']) or (cdcr_num in voced_credit['Cdcno']):
    programming.append('Yes')
  else:
    programming.append('No')
  # Rule violation reports
  ext = rv_report[rv_report['CDCR\nNumber'] == cdcr_num][['Rule Violation Date', 'Division', 'Rule Violation']].reset_index(drop = True).to_dict('index')
  rvr.append("\n\n".join("\n".join(k_b + ': ' + str(v_b) for k_b, v_b in v_a.items()) for k_a, v_a in ext.items()))

# Store lists in dataframe
df['Current Convictions'] = current_conv
df['Prior Convictions'] = prior_conv
df['Programming'] = programming
df['Rules Violations'] = rvr

# Write data to excel files
df.to_excel(write_path, index = False)